# Create Monthly Task Reports in Smartsheet


In [1]:
%load_ext nb_black

import os
import json
import logging
from datetime import datetime, timedelta
from typing import Dict, List

import pandas as pd
import numpy as np
import prefect
from box import Box

import smartsheet

# uses the pretty okay SDK here: https://github.com/ProdPerfect/monday
from monday import MondayClient
from mondaydotcom_utils.formatted_value import FormattedValue, get_col_defs
from mondaydotcom_utils.time_block import TimeBlock
from mondaydotcom_utils.utilities import (
    breakout_record,
    get_items_by_board,
    validate_task_record,
)
from prefect import Flow, Parameter, task, unmapped
from prefect.executors import LocalDaskExecutor, LocalExecutor

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

<IPython.core.display.Javascript object>

In [2]:
TASKS_BOARD_ID = "1883170887"
AGREEMENTS_BOARD_ID = "1882423671"
PROJECTS_BOARD_ID = "1882404316"
ACCOUNTS_BOARD_ID = "1882424009"

PROJECT_TASK_TIME_BOARD_ID = "2398200403"

# don't set this here for development work... use the secrets-<environment>.yaml files instead.
MONDAY_KEY = ""
SMARTSHEET_KEY = ""
environment = "dev"

# change these or set as papermill parameters to report on year and month
year_for_report = 2022
month_for_report = 3

<IPython.core.display.Javascript object>

In [3]:
if not MONDAY_KEY:
    # key hasn't been passed as a papermill parameter... get it from a file?
    secrets = Box.from_yaml(filename=f"secrets-{environment}.yaml")
    MONDAY_KEY = secrets.apps.monday.API_KEY

if not SMARTSHEET_KEY:
    # key hasn't been passed as a papermill parameter... get it from a file?
    secrets = Box.from_yaml(filename=f"secrets-{environment}.yaml")
    SMARTSHEET_KEY = secrets.apps.smartsheet.API_KEY

<IPython.core.display.Javascript object>

In [4]:
# connect monday client
conn = MondayClient(MONDAY_KEY)

<IPython.core.display.Javascript object>

In [5]:
# connect smartsheet client
ss_client = smartsheet.Smartsheet(SMARTSHEET_KEY)
ss_client.errors_as_exceptions(True)

<IPython.core.display.Javascript object>

In [6]:
users = conn.users.fetch_users()["data"]["users"]
users_df = pd.DataFrame(users).set_index("id")
users_df.head()

,name,email,enabled,teams
id,,,,
25810257,Steve Taylor,stephen.taylor@cuanschutz.edu,True,[]
25815853,Faisal Alquaddoomi,faisal.alquaddoomi@cuanschutz.edu,True,[]
25815860,Vincent Rubinetti,vincent.rubinetti@cuanschutz.edu,True,[]
26327954,Audrey Wen,audrey.wen@cuanschutz.edu,True,[]
27773472,timothy.putman@cuanschutz.edu,timothy.putman@cuanschutz.edu,True,[]


<IPython.core.display.Javascript object>

In [7]:
def breakdown_status(x):
    # use this to break down the status columns
    # TODO move this to mondaydotcom-utils in the formatters

    my_list = []
    json1 = json.loads(x)

    if json1.get("text"):
        my_list.append(json1["text"])
    if json1.get("changed_at"):
        my_list.append(json1["changed_at"])

    return ";".join(my_list)

<IPython.core.display.Javascript object>

In [8]:
accounts_df = get_items_by_board(conn, ACCOUNTS_BOARD_ID)

accounts_df.rename(
    columns={"monday_id": "account_id", "Title": "Client Name"},
    inplace=True,
)

accounts_df.drop(
    columns=[
        "Contacts",
        "Item ID",
        "Subitems",
        "Notes",
        "Customer Projects",
        "Agreements",
        "Type",
    ],
    inplace=True,
)

accounts_df

,account_id,Client Name
0,1882439999,HealthAI: Admin & Operations
1,1882462147,CU SOM: IT Department
2,1882588856,CIDA: Center for Innovative Design & Analysis
3,1882681138,HealthAI: Greene Lab
4,1882681714,HealthAI: TISLab
5,1883644776,HealthAI: Bennett Lab
6,1883648098,HealthAI: Hunter Lab
7,1883649981,HealthAI: Way Lab
8,1907269862,HealthAI: Sean Davis
9,2246385174,HealthAI: Dwork Lab


<IPython.core.display.Javascript object>

In [9]:
projects_df = get_items_by_board(conn, PROJECTS_BOARD_ID)

projects_df.rename(
    columns={
        "monday_id": "project_id",
    },
    inplace=True,
)

projects_df.drop(
    columns=[
        "Repo Description (mirror)",
        "Project Tasks",
        "Subitems",
        "Etimated Time (Hours) (mirror)",
        "Total Task Time (Hours) (mirror)",
        "Project Contacts",
        "SET Resource",
        "Timeline",
        "Customer Source",
        "Tasks Status (mirror)",
        "Dependency",
        "Date Added",
        "Time Balance (Hours) (formula)",
        "Agreement NTE Hours (mirror)",
        "Timeline Days",
        "Item ID",
        "Project Health",
        "Notes",
        "Agreements",
    ],
    inplace=True,
)

projects_df["Project Lifecycle"] = projects_df["Project Lifecycle"].apply(
    breakdown_status
)

projects_df = projects_df.explode(["Account"], ignore_index=True)
projects_df

,project_id,Title,Account,Grant Number,Project Closed Date,Project Lifecycle
0,1882442059,TISLab: Monarch UI (3.0) Redesign,1882681714,213359,None,Open;2022-03-30T21:12:11.038Z
1,1882712838,Greenelab: lab-website-template and related si...,1882681138,None,2022-04-06,Closed;2022-04-06T12:36:51.968Z
2,1882738595,Greenelab: mygeneset.info,1882681138,None,None,2022-03-30T22:55:50.757Z
3,1882739627,CHAI: Manubot next-gen,1882439999,213269,None,2022-03-30T22:55:53.007Z
4,1882752029,"HealthAI ""lab"" Portfolio Site",1882439999,None,None,Open;2022-03-30T21:19:53.022Z
5,1882913862,TISLab: Graph DB Deployer,1882681714,None,2021-12-16,Closed;2022-03-30T21:12:38.875Z
6,1888314634,CHAI: Admin Technology Foundation,1882439999,None,None,Open;2022-03-30T21:19:49.920Z
7,1892630899,TISLab: Monarch GCP migration,1882681714,None,2022-03-30,Closed;2022-03-30T22:50:35.736Z
8,1957293587,Way: Grant Support,1883649981,None,None,Closed;2022-03-30T21:12:31.743Z
9,1969468997,"Greenelab: Biomedical Literature ""Word Lapse"" ...",1882681138,213269,None,Open;2022-03-30T21:12:58.873Z


<IPython.core.display.Javascript object>

In [10]:
# add the account to the projects
projects_df = pd.merge(
    projects_df, accounts_df, how="left", left_on="Account", right_on="account_id"
).drop(columns=["account_id"])
projects_df

,project_id,Title,Account,Grant Number,Project Closed Date,Project Lifecycle,Client Name
0,1882442059,TISLab: Monarch UI (3.0) Redesign,1882681714,213359,None,Open;2022-03-30T21:12:11.038Z,HealthAI: TISLab
1,1882712838,Greenelab: lab-website-template and related si...,1882681138,None,2022-04-06,Closed;2022-04-06T12:36:51.968Z,HealthAI: Greene Lab
2,1882738595,Greenelab: mygeneset.info,1882681138,None,None,2022-03-30T22:55:50.757Z,HealthAI: Greene Lab
3,1882739627,CHAI: Manubot next-gen,1882439999,213269,None,2022-03-30T22:55:53.007Z,HealthAI: Admin & Operations
4,1882752029,"HealthAI ""lab"" Portfolio Site",1882439999,None,None,Open;2022-03-30T21:19:53.022Z,HealthAI: Admin & Operations
5,1882913862,TISLab: Graph DB Deployer,1882681714,None,2021-12-16,Closed;2022-03-30T21:12:38.875Z,HealthAI: TISLab
6,1888314634,CHAI: Admin Technology Foundation,1882439999,None,None,Open;2022-03-30T21:19:49.920Z,HealthAI: Admin & Operations
7,1892630899,TISLab: Monarch GCP migration,1882681714,None,2022-03-30,Closed;2022-03-30T22:50:35.736Z,HealthAI: TISLab
8,1957293587,Way: Grant Support,1883649981,None,None,Closed;2022-03-30T21:12:31.743Z,HealthAI: Way Lab
9,1969468997,"Greenelab: Biomedical Literature ""Word Lapse"" ...",1882681138,213269,None,Open;2022-03-30T21:12:58.873Z,HealthAI: Greene Lab


<IPython.core.display.Javascript object>

In [11]:
def breakout_time_sessions(row):
    """
    Break down the Monday.com time structure into something simpler for us.

    This is used with a DataFrame.apply()
    """

    mct = TimeBlock()
    mct.parse(row["Actual Time"])
    return mct.total_duration_hours, mct.time_records

<IPython.core.display.Javascript object>

In [12]:
# only getting done tasks
tasks_df = get_items_by_board(conn, TASKS_BOARD_ID, "status", "Done")

tasks_df.rename(
    columns={
        "monday_id": "task_id",
    },
    inplace=True,
)

tasks_df.drop(
    columns=[
        "Subtasks",
        "Timeline Hours (Estimated) (formula)",
        "Total Actual Hours (formula)",
        "Customer Repos",
        "Billing Agreement",
        "Project Lifecycle (mirror)",
        "Projected Hours Remaining (formula)",
    ],
    inplace=True,
)

# break the time sessions out
tasks_df[["Total Duration Hours", "Time Sessions"]] = tasks_df.apply(
    breakout_time_sessions, axis=1, result_type="expand"
)

# Only include Ready tasks
tasks_df = tasks_df.loc[
    tasks_df["Integration Message"].str.startswith("Ready", na=False)
]

# projects should be limited to just one, so this will bring it out of the list
tasks_df = tasks_df.explode(["Customer Project"], ignore_index=True)
tasks_df.head()

,task_id,Title,Actual Hours,Actual Time,Customer Project,Date Added,Date Completed,Dependencies,Integration Message,Issue URL,Notes,Owner,Pull Request URL,Status,Timeline,Timeline Days,Total Duration Hours,Time Sessions
0,1987638878,automating import of wakatime hours into monda...,NaN,"{""running"":false,""duration"":21180,""startDate"":...",2208602434,None,2022-04-06,None,Ready - 2022-04-25 16:21:50.783977,None,unfortunately after implementing the wakatime ...,"[{'id': 25815853, 'kind': 'person'}]",None,"{""text"": ""Done"", ""changed_at"": ""2022-04-06T11:...","{""to"":""2021-12-07"",""from"":""2021-12-06"",""change...",NaN,5.883333,"[{'owner_id': 25815853, 'manually_entered': Tr..."
1,2249793370,node page header,NaN,"{""running"":false,""duration"":10800,""startDate"":...",1882442059,None,2022-03-24,None,Ready - 2022-04-25 16:22:12.573489,None,https://github.com/monarch-initiative/monarch-...,"[{'id': 25815860, 'kind': 'person'}]",None,"{""text"": ""Done"", ""changed_at"": ""2022-03-24T15:...","{""to"":""2022-03-19"",""from"":""2022-03-19"",""visual...",1,3.000000,"[{'owner_id': 25815860, 'manually_entered': Tr..."
2,2249793382,node page table of contents/navigator,NaN,"{""running"":false,""duration"":46800,""startDate"":...",1882442059,None,2022-03-24,None,Ready - 2022-04-25 16:22:18.526148,None,https://github.com/monarch-initiative/monarch-...,"[{'id': 25815860, 'kind': 'person'}]",None,"{""text"": ""Done"", ""changed_at"": ""2022-03-24T15:...","{""to"":""2022-03-21"",""from"":""2022-03-19"",""visual...",3,13.000000,"[{'owner_id': 25815860, 'manually_entered': Tr..."
3,2249793395,node page overview and details sections,NaN,"{""running"":false,""duration"":57600,""startDate"":...",1882442059,None,2022-03-30,None,Ready - 2022-04-25 16:22:23.152543,None,None,"[{'id': 25815860, 'kind': 'person'}]",https://github.com/monarch-initiative/monarch-...,"{""text"": ""Done"", ""changed_at"": ""2022-03-30T15:...","{""to"":""2022-03-26"",""from"":""2022-03-25"",""visual...",2,16.000000,"[{'owner_id': 25815860, 'manually_entered': Tr..."
4,2249793413,node page hierarchy section,NaN,"{""running"":false,""duration"":28800,""startDate"":...",1882442059,None,2022-03-30,None,Ready - 2022-04-25 16:22:28.989135,None,None,"[{'id': 25815860, 'kind': 'person'}]",https://github.com/monarch-initiative/monarch-...,"{""text"": ""Done"", ""changed_at"": ""2022-03-30T15:...","{""to"":""2022-04-05"",""from"":""2022-04-05"",""visual...",1,8.000000,"[{'owner_id': 25815860, 'manually_entered': Tr..."


<IPython.core.display.Javascript object>

Break down the Monday.com session items into individual "journal tasks".

In [13]:
journal_items = []
records = tasks_df.to_dict(orient="records")
for record in records:
    new_list = breakout_record(record, users_df)

    # go through those N records, one by one
    for item in new_list:
        journal_items.append(item)

journal_task_df = pd.DataFrame(journal_items)

# break out the actual task status also; we've already used the changed_at
# field to help break the records out... so this can be simplified for info
journal_task_df["task_status"] = journal_task_df["Status"].apply(
    lambda x: json.loads(x)["text"] if x else None
)

# convert to a dataframe date... a bit crude for filtering
journal_task_df["task_end_date"] = pd.to_datetime(journal_task_df["task_end_date"])
journal_task_df["task_end_year"] = pd.DatetimeIndex(
    journal_task_df["task_end_date"]
).year
journal_task_df["task_end_month"] = pd.DatetimeIndex(
    journal_task_df["task_end_date"]
).month

journal_task_df.head()

,task_id,Title,Actual Hours,Actual Time,Customer Project,Date Added,Date Completed,Dependencies,Integration Message,Issue URL,...,Timeline Days,Total Duration Hours,Time Sessions,owner,hours,task_end_date,integration_state_rule,task_status,task_end_year,task_end_month
0,1987638878,automating import of wakatime hours into monda...,NaN,"{""running"":false,""duration"":21180,""startDate"":...",2208602434,None,2022-04-06,None,Ready - 2022-04-25 16:21:50.783977,None,...,NaN,5.883333,"[{'owner_id': 25815853, 'manually_entered': Tr...",Faisal Alquaddoomi,5.883333,2022-04-06 00:00:00+00:00,hours_from_session_records,Done,2022,4
1,2249793370,node page header,NaN,"{""running"":false,""duration"":10800,""startDate"":...",1882442059,None,2022-03-24,None,Ready - 2022-04-25 16:22:12.573489,None,...,1.0,3.000000,"[{'owner_id': 25815860, 'manually_entered': Tr...",Vincent Rubinetti,3.000000,2022-03-24 00:00:00+00:00,hours_from_session_records,Done,2022,3
2,2249793382,node page table of contents/navigator,NaN,"{""running"":false,""duration"":46800,""startDate"":...",1882442059,None,2022-03-24,None,Ready - 2022-04-25 16:22:18.526148,None,...,3.0,13.000000,"[{'owner_id': 25815860, 'manually_entered': Tr...",Vincent Rubinetti,5.000000,2022-03-24 00:00:00+00:00,hours_from_session_records,Done,2022,3
3,2249793382,node page table of contents/navigator,NaN,"{""running"":false,""duration"":46800,""startDate"":...",1882442059,None,2022-03-24,None,Ready - 2022-04-25 16:22:18.526148,None,...,3.0,13.000000,"[{'owner_id': 25815860, 'manually_entered': Tr...",Vincent Rubinetti,8.000000,2022-03-24 00:00:00+00:00,hours_from_session_records,Done,2022,3
4,2249793395,node page overview and details sections,NaN,"{""running"":false,""duration"":57600,""startDate"":...",1882442059,None,2022-03-30,None,Ready - 2022-04-25 16:22:23.152543,None,...,2.0,16.000000,"[{'owner_id': 25815860, 'manually_entered': Tr...",Vincent Rubinetti,8.000000,2022-03-30 00:00:00+00:00,hours_from_session_records,Done,2022,3


<IPython.core.display.Javascript object>

In [14]:
# only interested in this month's tasks
mask = (journal_task_df["task_end_year"] == year_for_report) & (
    journal_task_df["task_end_month"] == month_for_report
)
journal_task_df = journal_task_df.loc[mask]

# Some of this helped build the record, some is just mirror or lookup gak.
journal_task_df.drop(
    columns=[
        "Actual Hours",
        "Actual Time",
        "Customer Repos",
        "Date Added",
        "Date Completed",
        "Dependencies",
        "Integration Message",
        "Subtasks",
        "Timeline",
        "Total Actual Hours (formula)",
        "Total Duration Hours",
        "Time Sessions",
        "Owner",
        "Status",
        "Timeline Hours (Estimated) (formula)",
        "Project Lifecycle (mirror)",
        "Projected Hours Remaining (formula)",
        "Billing Agreement",
        "Timeline Days",
        "task_status",
    ],
    inplace=True,
    errors="ignore",
)

journal_task_df

,task_id,Title,Customer Project,Issue URL,Notes,Pull Request URL,owner,hours,task_end_date,integration_state_rule,task_end_year,task_end_month
1,2249793370,node page header,1882442059,None,https://github.com/monarch-initiative/monarch-...,None,Vincent Rubinetti,3.0,2022-03-24 00:00:00+00:00,hours_from_session_records,2022,3
2,2249793382,node page table of contents/navigator,1882442059,None,https://github.com/monarch-initiative/monarch-...,None,Vincent Rubinetti,5.0,2022-03-24 00:00:00+00:00,hours_from_session_records,2022,3
3,2249793382,node page table of contents/navigator,1882442059,None,https://github.com/monarch-initiative/monarch-...,None,Vincent Rubinetti,8.0,2022-03-24 00:00:00+00:00,hours_from_session_records,2022,3
4,2249793395,node page overview and details sections,1882442059,None,None,https://github.com/monarch-initiative/monarch-...,Vincent Rubinetti,8.0,2022-03-30 00:00:00+00:00,hours_from_session_records,2022,3
5,2249793395,node page overview and details sections,1882442059,None,None,https://github.com/monarch-initiative/monarch-...,Vincent Rubinetti,8.0,2022-03-30 00:00:00+00:00,hours_from_session_records,2022,3
...,...,...,...,...,...,...,...,...,...,...,...,...
79,2512879141,"rancher learning, setup",1888314634,None,None,None,Faisal Alquaddoomi,5.0,2022-03-30 00:00:00+00:00,hours_from_session_records,2022,3
80,2512879141,"rancher learning, setup",1888314634,None,None,None,Faisal Alquaddoomi,1.0,2022-03-30 00:00:00+00:00,hours_from_session_records,2022,3
81,2512879141,"rancher learning, setup",1888314634,None,None,None,Faisal Alquaddoomi,6.0,2022-03-30 00:00:00+00:00,hours_from_session_records,2022,3
82,2512879141,"rancher learning, setup",1888314634,None,None,None,Faisal Alquaddoomi,5.0,2022-03-30 00:00:00+00:00,hours_from_session_records,2022,3


<IPython.core.display.Javascript object>

Finally merge the tasks and projects together for a final task list.

In [15]:
def month_end_date(year, month):
    """Calculate the month end date given a year and month."""
    month += 1
    if month == 13:
        month = 1
        year += 1

    tempdate = datetime.strptime(f"{year}-{month}-1", "%Y-%m-%d")
    return (tempdate - timedelta(days=1)).strftime("%m/%d/%Y")

<IPython.core.display.Javascript object>

In [16]:
def month_end_me(row):
    return month_end_date(row["task_end_year"], row["task_end_month"])

<IPython.core.display.Javascript object>

In [17]:
df = pd.merge(
    journal_task_df,
    projects_df,
    how="left",
    left_on="Customer Project",
    right_on="project_id",
)

df["Month Ending Date"] = df.apply(month_end_me, axis=1)

df.rename(
    columns={
        "monday_id_x": "monday_id",
        "monday_id_y": "project_id",
        "Title_x": "Title",
        "Title_y": "Project Title",
        "Notes_x": "Notes",
        "Notes_y": "Project Notes",
        "hours": "Hours",
        "owner": "Resource",
    },
    inplace=True,
)

df.drop(
    columns=["project_id", "monday_id", "task_end_year", "task_end_month", "Account"],
    inplace=True,
    errors="ignore",
)


df

,task_id,Title,Customer Project,Issue URL,Notes,Pull Request URL,Resource,Hours,task_end_date,integration_state_rule,Project Title,Grant Number,Project Closed Date,Project Lifecycle,Client Name,Month Ending Date
0,2249793370,node page header,1882442059,None,https://github.com/monarch-initiative/monarch-...,None,Vincent Rubinetti,3.0,2022-03-24 00:00:00+00:00,hours_from_session_records,TISLab: Monarch UI (3.0) Redesign,213359,None,Open;2022-03-30T21:12:11.038Z,HealthAI: TISLab,03/31/2022
1,2249793382,node page table of contents/navigator,1882442059,None,https://github.com/monarch-initiative/monarch-...,None,Vincent Rubinetti,5.0,2022-03-24 00:00:00+00:00,hours_from_session_records,TISLab: Monarch UI (3.0) Redesign,213359,None,Open;2022-03-30T21:12:11.038Z,HealthAI: TISLab,03/31/2022
2,2249793382,node page table of contents/navigator,1882442059,None,https://github.com/monarch-initiative/monarch-...,None,Vincent Rubinetti,8.0,2022-03-24 00:00:00+00:00,hours_from_session_records,TISLab: Monarch UI (3.0) Redesign,213359,None,Open;2022-03-30T21:12:11.038Z,HealthAI: TISLab,03/31/2022
3,2249793395,node page overview and details sections,1882442059,None,None,https://github.com/monarch-initiative/monarch-...,Vincent Rubinetti,8.0,2022-03-30 00:00:00+00:00,hours_from_session_records,TISLab: Monarch UI (3.0) Redesign,213359,None,Open;2022-03-30T21:12:11.038Z,HealthAI: TISLab,03/31/2022
4,2249793395,node page overview and details sections,1882442059,None,None,https://github.com/monarch-initiative/monarch-...,Vincent Rubinetti,8.0,2022-03-30 00:00:00+00:00,hours_from_session_records,TISLab: Monarch UI (3.0) Redesign,213359,None,Open;2022-03-30T21:12:11.038Z,HealthAI: TISLab,03/31/2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,2512879141,"rancher learning, setup",1888314634,None,None,None,Faisal Alquaddoomi,5.0,2022-03-30 00:00:00+00:00,hours_from_session_records,CHAI: Admin Technology Foundation,None,None,Open;2022-03-30T21:19:49.920Z,HealthAI: Admin & Operations,03/31/2022
63,2512879141,"rancher learning, setup",1888314634,None,None,None,Faisal Alquaddoomi,1.0,2022-03-30 00:00:00+00:00,hours_from_session_records,CHAI: Admin Technology Foundation,None,None,Open;2022-03-30T21:19:49.920Z,HealthAI: Admin & Operations,03/31/2022
64,2512879141,"rancher learning, setup",1888314634,None,None,None,Faisal Alquaddoomi,6.0,2022-03-30 00:00:00+00:00,hours_from_session_records,CHAI: Admin Technology Foundation,None,None,Open;2022-03-30T21:19:49.920Z,HealthAI: Admin & Operations,03/31/2022
65,2512879141,"rancher learning, setup",1888314634,None,None,None,Faisal Alquaddoomi,5.0,2022-03-30 00:00:00+00:00,hours_from_session_records,CHAI: Admin Technology Foundation,None,None,Open;2022-03-30T21:19:49.920Z,HealthAI: Admin & Operations,03/31/2022


<IPython.core.display.Javascript object>

In [18]:
# create a group by report and post to SE Project/Grant Time smartsheet
report_df = (
    df.groupby(["Client Name", "Project Title", "Resource"])
    .agg(
        {
            "Hours": "sum",
            "Month Ending Date": "first",
            "Grant Number": "first",
        }
    )
    .reset_index()
)
report_df

,Client Name,Project Title,Resource,Hours,Month Ending Date,Grant Number
0,HealthAI: Admin & Operations,CHAI: Admin Technology Foundation,Faisal Alquaddoomi,55.15,03/31/2022,None
1,HealthAI: Admin & Operations,CHAI: Admin Technology Foundation,Steve Taylor,58.00,03/31/2022,None
2,HealthAI: Admin & Operations,CHAI: Center/Department Reviews 2021,Steve Taylor,8.00,03/31/2022,None
3,HealthAI: Greene Lab,"Greenelab: Biomedical Literature ""Word Lapse"" ...",Faisal Alquaddoomi,84.50,03/31/2022,213269
4,HealthAI: Greene Lab,"Greenelab: Biomedical Literature ""Word Lapse"" ...",Vincent Rubinetti,20.00,03/31/2022,213269
5,HealthAI: TISLab,TISLab: Monarch GCP migration,Faisal Alquaddoomi,0.00,03/31/2022,None
6,HealthAI: TISLab,TISLab: Monarch UI (3.0) Redesign,Faisal Alquaddoomi,6.00,03/31/2022,213359
7,HealthAI: TISLab,TISLab: Monarch UI (3.0) Redesign,Vincent Rubinetti,56.00,03/31/2022,213359
8,HealthAI: TISLab,TISLab: Staffing/Support 2021,Faisal Alquaddoomi,12.00,03/31/2022,None


<IPython.core.display.Javascript object>

In [19]:
def filter_tasks(df, client, project, resource, month_end_date):
    """Get the journal tasks based on the details we'll send to smartsheet."""
    return df[
        (
            (df["Client Name"] == client)
            & (df["Project Title"] == project)
            & (df["Resource"] == resource)
            & (df["Month Ending Date"] == month_end_date)
        )
    ]

<IPython.core.display.Javascript object>

Now, Smartsheet's turn?

In [20]:
sheet_name = "SE Project/Grant Time"

search_results = ss_client.Search.search(sheet_name).results

# helpful: https://stackoverflow.com/questions/52065527/python-best-way-to-get-smartsheet-sheet-by-name
time_sheet_id = next(
    result.object_id for result in search_results if result.object_type == "sheet"
)
time_sheet = ss_client.Sheets.get_sheet(time_sheet_id)

<IPython.core.display.Javascript object>

In [21]:
# break down the cell IDs into a quick lookup box
cell_ids = {}
for column in time_sheet.columns:
    my_column = column.to_dict()
    cell_ids[my_column["title"]] = my_column["id"]
cell_ids

{'Month-end Date': 8924238069426052,
 'Project Title': 7231746076895108,
 'Resource': 128145047218052,
 'Completed Hours': 4631744674588548,
 'Notes': 4420638442055556,
 'Account/Client': 2815554847303556,
 'Grant Proposal #': 2743273173346180}

<IPython.core.display.Javascript object>

Add the records to Smartsheet

In [22]:
rows = []
for k, v in report_df.to_dict("index").items():

    row = ss_client.models.row.Row()

    row.cells.append(
        {"column_id": cell_ids["Account/Client"], "value": v["Client Name"]}
    )
    if v.get("Grant Number"):
        row.cells.append(
            {"column_id": cell_ids["Grant Proposal #"], "value": v["Grant Number"]}
        )
    row.cells.append(
        {"column_id": cell_ids["Project Title"], "value": v["Project Title"]}
    )
    row.cells.append(
        {"column_id": cell_ids["Month-end Date"], "value": v["Month Ending Date"]}
    )
    row.cells.append({"column_id": cell_ids["Completed Hours"], "value": v["Hours"]})
    row.cells.append({"column_id": cell_ids["Resource"], "value": v["Resource"]})

    row.to_bottom = True
    rows.append(row)


result = ss_client.Sheets.add_rows(time_sheet_id, rows)

<IPython.core.display.Javascript object>

In [23]:
# get the row ids and create a Series
my_list = []
for row in result.to_dict()["data"]:
    my_list.append(row["id"])


row_series = pd.Series(my_list, name="row_id", dtype=np.int64)
row_series

report_df = pd.concat([report_df, row_series], axis=1)
report_df

NameError: name 'result' is not defined

<IPython.core.display.Javascript object>

In [ ]:
# create a dictionary to make attaching the files easier
new_dict = {}
for k, v in report_df.to_dict("index").items():
    row_id = v["row_id"]

    # add the filtered tasks to a list
    new_dict[row_id] = filter_tasks(
        df,
        v["Client Name"],
        v["Project Title"],
        v["Resource"],
        v["Month Ending Date"],
    )
new_dict

In [ ]:
# attach file to each record
if not os.path.exists("_cache"):
    os.mkdir("_cache")

for k, v in new_dict.items():
    filename = os.path.join("_cache", f"{k}.csv")

    # save file
    v.to_csv(filename, index=False)

    with open(filename, "r") as f:
        ss_client.Attachments.attach_file_to_row(time_sheet_id, k, f)

In [26]:
for k, v in df.to_dict("index").items():
    result = conn.items.change_item_value(
        TASKS_BOARD_ID, v["task_id"], "text01", f"Posted - {datetime.now()}"
    )

<IPython.core.display.Javascript object>